In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, div.text_cell_render ol li p, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# ※ Quiz : 경주여행과 전주여행에 대해 최빈단어시각화와 유사도분석
- (1) naver open API를 활용하여 네이버지식인 "전주여행", "경주여행"를 각각 500건씩 검색 백업(naver.csv)
    * replace(), re.sum()이용
    * 백업 파일 결과 : query,no,title,link,description,total_text(title+' '+description)
- (2) 품사태깅 백업(naver_pos.csv)
    * query, no, token, pos
- (3) 명사만 추출 백업(naver_pos_nouns.csv)
    * query, token, pos
- (4) 빈도분석 백업(naver_pos_nouns_count.csv)
    * token, 경주빈도, 전주빈도, 빈도합
- (5) 빈도 시각화(워드클라우드, Text.plot)
    * 워드클라우드 이미지 저장
- (6) 단어간 유사도 분석(Word2Vec, 연관분석)

# 1. naver open API를 활용하여 검색어 500건 추출
- query,no,title,link,description,total_text(title+' '+description)

In [3]:
# .env 가져오기(방법1) - 추천
from dotenv import load_dotenv
import os
load_dotenv()
# print(os.getenv('CLIENT_ID'))
# print(os.getenv('CLIENT_SECRET'))

True

In [5]:
# .env 가져오기(방법2)
from decouple import config
# print(config('CLIENT_ID'))
# print(config('CLIENT_SECRET'))

In [8]:
# 네이버 검색 API 예제 - 블로그 검색
import os
import sys
import urllib.request
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = urllib.parse.quote("경주 여행")
url = "https://openapi.naver.com/v1/search/blog?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8')[:200])
else:
    print("Error Code:" + rescode)

{
	"lastBuildDate":"Tue, 02 Dec 2025 14:39:13 +0900",
	"total":1003198,
	"start":1,
	"display":10,
	"items":[
		{
			"title":"내 마음대로 기록하는 <b>경주여행<\/b> 2박3일",
			"link":"https:\/\/blog.naver.com\/hello


In [11]:
# 방법1
import requests
import pandas as pd
import json # response텍스트를 json스타일의 딕셔너리로
import re  # 정규표현식
from html import unescape # description에 &lt;등의 특수문자를 <(문자)로 변경

client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
query = '경주 여행'
start = 1
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
# url = f'https://openapi.naver.com/v1/search/kin?query={query}&display=100&start={start}'
# response = requests.get(url, headers=headers)
url = 'https://openapi.naver.com/v1/search/kin'
params = {'query':query, 'display':100, 'start':start}
response = requests.get(url, params=params, headers=headers)
# print(response.text[:500])
# items = json.loads(response.text)['items']
items = response.json()['items']
print(len(items))

100

In [20]:
# title, link, description 추출(b태그삭제, 특수문자조정(&gt; -> >), 정규표현식을 이용한 한글추출 )
item = items[97]
title = item.get('title').replace('<b>', '').replace('</b>', '')
link = item['link']
description = item.get('description').replace('<b>', '').replace('</b>', '')
# html 특수 문자 처리
description = unescape(description)
# 정규표현식 : 알파벳, 숫자, 한글만 남기고 나머지는 제거
description = re.sub(r'[^a-zA-Z0-9가-힣]', ' ', description)
description = re.sub('\s+', ' ', description)
print(title, link, description)

경주 여행지 추천 https://kin.naver.com/qna/detail.naver?d1id=9&dirId=90111&docId=261370795&qb=6rK97KO8IOyXrO2WiQ==&enc=utf8  2 부산을 벗어난 경주 여행지 알려주세요 3 당일치기로 경주 여행코스 밑에 자세하게 권역별로 여행코스를 소개했습니다 경주여행이 처음이라면 대릉원 첨성대 참고로 쭈니아빠 블로그를 방문하시면 경주여행 


In [23]:
def get_search_element_return(query, start):
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    # url = f'https://openapi.naver.com/v1/search/kin?query={query}&display=100&start={start}'
    # response = requests.get(url, headers=headers)
    url = 'https://openapi.naver.com/v1/search/kin'
    params = {'query':query, 'display':100, 'start':start}
    response = requests.get(url, params=params, headers=headers)
    # print(response.text[:500])
    # items = json.loads(response.text)['items']
    items = response.json()['items']
    result = []# 검색한 정보를 append할 리스트
    for i, item in enumerate(items):
        title = item.get('title').replace('<b>', ' ').replace('</b>', ' ')
        link  = item.get('link')
        description = item.get('description').replace('<b>', ' ').replace('</b>', ' ')
        # 특수문자처리
        description = unescape(description)
        # 영문자, 숫자, 한글을 제외한 문자 제거
        description = re.sub(r'[^a-zA-Z0-9가-힣]', ' ', description)
        description = re.sub('\s+', ' ', description)
        result.append({
            'query':query,
            'no':(start-1)*100 + i+1
        })

In [24]:
get_search_element_return("경주 여행", 1)

0 APEC 기간 <b>경주여행</b>
1 <b>경주 여행</b> 강수량 45% 비 오는거겠죠 ?
2 6~7월 <b>경주 여행</b>에서 꼭 가봐야 할 명소는?
3 <b>경주 여행</b> 시 황리단길에서 꼭 방문해야 할 카페와 맛집... 
4 <b>경주 여행</b> 중 꼭 가봐야 할 맛집 리스트는 무엇인가요?
5 어머니랑 1박2일 <b>경주여행</b>
6 <b>경주 여행</b>을 가는데 갈만한 곳 추천좀
7 <b>경주 여행</b> 맛집
8 <b>경주 여행</b>
9 <b>경주 여행</b> 중 꼭 가봐야 할 맛집 리스트는 무엇인가요?
10 <b>경주여행</b> 30만원
11 [내공100] <b>경주여행</b> 꼭 가야하는 곳!!!!!
12 <b>경주 여행</b>
13 <b>경주 여행</b> 추천
14 <b>경주 여행</b> 중 아나타 카페 방문, 어떤 분위기였나요?
15 <b>경주여행</b> 숙소
16 6-7월 <b>경주 여행</b> 대릉원 외 꼭 방문해야 할 명소는?
17 <b>경주 여행</b> 시 황리단길 주차 정보와 주변 관광지 연계... 
18 <b>경주여행</b>지 2박3일 추천
19 <b>경주 여행</b>코스
20 2박3일 <b>경주 여행</b>
21 1박2일 <b>경주 여행</b> 일정 짜주세요
22 <b>경주여행</b> 코스 동선 어떻게 짜야될까요?
23 엄마와<b>경주여행</b>
24 초등학생<b>경주여행</b>
25 <b>경주 여행</b> 질문
26 가을때 가기 좋은 <b>경주 여행</b>지가 어디인가요?
27 <b>경주여행</b> 동선 부탁드려요
28 아이와 <b>경주여행</b>코스 짜려고 하는데 동선 ㅠㅠㅠ어렵네요
29 11월말 <b>경주여행</b>
30 <b>경주여행</b> 코스(지도),숙소
31 <b>경주여행</b>3박4일 갈려고하는데 24만원 괜찮을까요
32 2월 1~2일 <b>경주여행</b> 날씨
33 <b>경주여행</b> 실내관광
34 3월 1일 <b>경주 여행</b> 코스 (당일치기)
35 혼자 떠나는 <b>경주 여행</b>, 안